### Mediapipe 라이브러리를 사용한 얼굴, 동작 추출
- 구글에서 지원하는 AI 프레임워크
- 기능
  - 얼굴 검출
  - 얼굴 3D Mesh 검출 (400개가 넘는점)
  - 손 동작 인식
  - 사람 자세 검출 등

In [2]:
import mediapipe as mp
import cv2

#### Face Mesh 검출

In [14]:
# 얼굴 메시, 그리기 라이브러리 가져오기
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

# 그리기 설정 (선 두께, 원의 반지름)
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius = 1)

cap = cv2.VideoCapture('images/video_for_sticker.mp4')

# 최소 검출 신뢰도, 최소 추적 신뢰도
with mp_face_mesh.FaceMesh(min_detection_confidence=0.5, # 얼굴일 확률이 50퍼가 넘어가면 트래킹으로 넘어감
                           min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():
        ret, frame = cap.read() # 동영상의 한 프레임 읽기

        if not ret:
            print("이미지 읽기 실패 혹은 전체 영상 플레이 완료")
            break
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # 미디어파이프 처리를 위해 RGB로
        # 얼굴 메시 검출
        result = face_mesh.process(image)

        frame = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # CV로 올바르게 show하기 위해 다시 BGR로

        # 검출된 랜드마크와 랜드마크를 연결하는 선 그리기
        if result.multi_face_landmarks:
            for face_landmarks in result.multi_face_landmarks:
                mp_drawing.draw_landmarks(
                    image = frame,
                    landmark_list = face_landmarks,
                    connections = mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec = drawing_spec,
                    connection_drawing_spec = drawing_spec
                )

        cv2.imshow("video", frame)

        key = cv2.waitKey(33) # 키보드 입력값 변수에 담기

        if key == 49:  # 숫자 1
            break

cap.release()
cv2.destroyAllWindows()

#### 얼굴, 손, 포즈 검출

In [ ]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

drawing_spec1 = mp_drawing.DrawingSpec(thickness=1, color=(0, 0, 255))
drawing_spec2 = mp_drawing.DrawingSpec(thickness=3, color=(255, 0, 0))

cap = cv2.VideoCapture('images/all_body_video.mp4')

with mp_holistic.Holistic(min_detection_confidence = 0.5,
                          min_tracking_confidence = 0.5,
                          ) as holistic:
    while cap.isOpened():
        ret, frame = cap.read() # 동영상의 한 프레임 읽기

        if not ret:
            print("이미지 읽기 실패 혹은 전체 영상 플레이 완료")
            break

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)    
        result = holistic.process(image) # 얼굴, 포즈 등 검출
        frame = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(frame, result.pose_landmarks,
                                  mp_holistic.POSE_CONNECTIONS,
                                  landmark_drawing_spec = drawing_spec1,
                                  connection_drawing_spec = drawing_spec2,
                                  )

        cv2.imshow("video", frame)

        key = cv2.waitKey(33) # 키보드 입력값 변수에 담기

        if key == 49:  # 숫자 1
            break

cap.release()
cv2.destroyAllWindows()

#### Pose 활용 자세 판별

- 오프셋 거리 계산
  - 측면보기 상태에서 두 포인트 (눈, 어깨, 골반) 사이의 거리를 계산
  
\begin{align}
distance =  \sqrt{(x2 - x1)^2+(y2 - y1)^2}
\end{align}

In [ ]:
import math as m
import time
from PIL import ImageFont, ImageDraw, Image
import numpy as np

# 거리계산 함수
def findDistance(x1, y1, x2, y2):
    dist = m.sqrt((x2-x1)**2 + (y2-y1)**2)
    return dist

- 관심 선에서 y축으로 종속된 각도 계산

\begin{align}
\theta = \arccos (\frac{\vec{P_{12}}.\vec{P_{13}}}{|\vec{P_{12}}|.|\vec{P_{13}}|})
= \arccos (\frac{y_1^2 - y_1.y_2}{y_1\sqrt{(x_2 - x_1)^2 + (y_2 - y_1)^2}})
\end{align}

In [20]:
# 각도 계산 함수
def findAngle(x1, y1, x2, y2):
    theta = m.acos((y2-y1)*(-y1)/(m.sqrt(x2-x1)**2 + (y2-y1)**2)*y1)
    degree = int(180/m.pi)*theta
    return degree

In [21]:
# 폰트 설정
font = cv2.FONT_HERSHEY_SIMPLEX

# 색상 설정
blue = (255, 127, 0)
red = (50, 50, 255)
green = (127, 255, 0)
dark_blue = (127, 20, 0)
light_green = (127, 233, 100)
yellow = (0, 255, 255)
pink = (255, 0, 255)

# 프레임 카운터
good_frames = 0
bad_frames = 0

In [ ]:
cap = cv2.VideoCapture('images/holistic_ex.mp4')

while cap.isOpened():
    ret, frame = cap.read() # 동영상의 한 프레임 읽기

    if not ret:
        print("이미지 읽기 실패 혹은 전체 영상 플레이 완료")
        break

    cv2.imshow("video", frame)

    key = cv2.waitKey(20) # 키보드 입력값 변수에 담기

    if key == 49:  # 숫자 1
        break

cap.release()
cv2.destroyAllWindows()